# 1. Create a custom gym environment

In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
import math
from enum import Enum

In [2]:
class Action(Enum):
    UP = 0
    DOWN = 1
    LEFT = 2
    RIGHT = 3

In [71]:
class GameEnv(Env):
    def __init__(self):
        self.size = 10
        self.action_space = Discrete(4)
        self.observation_space = Box(low=0, high=2, shape=(self.size, self.size), dtype=np.int32)
        self.state, self.player = self.createBoard()
        self.game_length = 40
        
    def step(self, action):
        self.game_length -= 1
        done = False
        
        #evaluate move, save value of the new space before move then update the state
        invalid_move = False
        if Action(action) == Action.UP:
            new_pos = (self.player[0] -1, self.player[1])
            if new_pos[0] >= 0:
                new_space_val = self.movePlayer(new_pos)
            else:
                invalid_move = True
        elif Action(action) == Action.DOWN:
            new_pos = (self.player[0] +1, self.player[1])
            if new_pos[0] < self.size:
                new_space_val = self.movePlayer(new_pos)
            else:
                invalid_move = True
        elif Action(action) == Action.LEFT:
            new_pos = (self.player[0], self.player[1] -1)
            if new_pos[1] >= 0:
                new_space_val = self.movePlayer(new_pos)
            else:
                invalid_move = True
        elif Action(action) == Action.RIGHT:
            new_pos = (self.player[0], self.player[1] +1)
            if new_pos[1] < self.size:
                new_space_val = self.movePlayer(new_pos)
            else:
                invalid_move = True
        else:
            print("Invalid input to step function")
            
        #evaluate reward 
        reward = 0
        if(invalid_move):
            reward = -0.2
            done = False
        else:
            if new_space_val == 0:
                reward = -0.1
                done = False
            elif new_space_val == 2:
                reward = 20
                done = True
        
        #evaluate if out of time
        if self.game_length == 0:
            done = True
            reward = -20
            
        #placeholder for required return value
        info = {}
        
        return self.state, reward, done, info
    
    def movePlayer(self, new_pos):
        new_space_val = self.state[new_pos]
        self.state[self.player] = 0
        self.state[new_pos] = 1
        self.player = new_pos
        return new_space_val
        
    def render(self):
        pass
    
    def reset(self):
        self.state, self.player =  self.createBoard()
        self.game_length = 40
        return self.state
    
    def createBoard(self):
        board = np.zeros((self.size,self.size), dtype=np.int32)
        player_pos = (np.random.randint(self.size), np.random.randint(self.size))
        goal_pos = (np.random.randint(self.size), np.random.randint(self.size))
        player_goal_distance = math.sqrt((player_pos[0] - goal_pos[0])**2 + (player_pos[1] - goal_pos[1])**2)
        while player_goal_distance < self.size/2:
            goal_pos = (np.random.randint(self.size), np.random.randint(self.size))
            player_goal_distance = math.sqrt((player_pos[0] - goal_pos[0])**2 + (player_pos[1] - goal_pos[1])**2)
        board[player_pos] = 1
        board[goal_pos] = 2
        return board, player_pos

### Test

In [4]:
env = GameEnv()
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
        
    print(f'Episode:{episode} Score:{score}')

Episode:1 Score:-23.900000000000002
Episode:2 Score:-24.7
Episode:3 Score:-24.4
Episode:4 Score:-24.1
Episode:5 Score:-24.4
Episode:6 Score:-24.1
Episode:7 Score:-24.000000000000004
Episode:8 Score:-24.1
Episode:9 Score:-24.8
Episode:10 Score:-23.900000000000002


In [5]:
state = env.reset()
done = False
score = 0
while not done:
    action = env.action_space.sample()
    n_state, reward, done, info = env.step(action)
    score += reward
    print(env.render())

print(f'Episode:{episode} Score:{score}')

[[0 2 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
[[0 2 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
[[0 2 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
[[0 2 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
[[0 2 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0

# 2. Create Deep Learning Model

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

In [56]:
del model

In [63]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states[0],states[1])))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [64]:
states = env.observation_space.shape
actions = env.action_space.n
model = build_model(states,actions)

In [65]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 100)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 24)                2424      
_________________________________________________________________
dense_22 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_23 (Dense)             (None, 4)                 100       
Total params: 3,124
Trainable params: 3,124
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent

In [66]:
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
#from tf_agents.environments import tf_py_environment

In [67]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                   nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [68]:
#env =  tf_py_environment.TFPyEnvironment(env)

dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
    1/10000 [..............................] - ETA: 3:14:36 - reward: -0.2000

C:\Users\calho\anaconda3\envs\final_project\lib\site-packages\rl\memory.py:40: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 90s 9ms/step - reward: -0.6067 1s - reward: -0. - ETA
255 episodes - episode_reward: -23.793 [-27.800, 19.500] - loss: 6.306 - mae: 3.769 - mean_q: 5.047

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 53s 5ms/step - reward: -0.6105
253 episodes - episode_reward: -24.124 [-27.800, 19.400] - loss: 8.399 - mae: 5.487 - mean_q: 6.882

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 54s 5ms/step - reward: -0.5687
259 episodes - episode_reward: -21.960 [-27.800, 19.500] - loss: 5.727 - mae: 3.481 - mean_q: 1.528

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 55s 5ms/step - reward: -0.5465
260 episodes - episode_reward: -21.017 [-27.400, 19.500] - loss: 4.099 - mae: 3.540 - mean_q: -2.055

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 56s 6ms/step - reward: -0.5570
done, took 307.894 seconds


In [75]:
scores = dqn.test(env, nb_episodes=15, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 15 episodes ...
Episode 1: reward: -23.900, steps: 40
Episode 2: reward: -23.900, steps: 40
Episode 3: reward: -23.900, steps: 40
Episode 4: reward: -23.900, steps: 40
Episode 5: reward: -23.900, steps: 40
Episode 6: reward: -23.900, steps: 40
Episode 7: reward: -23.900, steps: 40
Episode 8: reward: -27.800, steps: 40
Episode 9: reward: -23.900, steps: 40
Episode 10: reward: -27.100, steps: 40
Episode 11: reward: -26.700, steps: 40
Episode 12: reward: -23.900, steps: 40
Episode 13: reward: -23.900, steps: 40
Episode 14: reward: -23.900, steps: 40
Episode 15: reward: -23.900, steps: 40
-24.559999999999995


# 4. Saving and Reloading Model

In [76]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [77]:
del model
del dqn
del env

In [80]:
env = GameEnv()
actions = env.action_space.n
states = env.observation_space.shape
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [81]:
dqn.load_weights('dqn_weights.h5f')

In [82]:
_ = dqn.test(env, nb_episodes=15, visualize=False)

Testing for 15 episodes ...
Episode 1: reward: -23.900, steps: 40
Episode 2: reward: -23.900, steps: 40
Episode 3: reward: -23.900, steps: 40
Episode 4: reward: -23.900, steps: 40
Episode 5: reward: -23.900, steps: 40
Episode 6: reward: -23.900, steps: 40
Episode 7: reward: -23.900, steps: 40
Episode 8: reward: -27.800, steps: 40
Episode 9: reward: -23.900, steps: 40
Episode 10: reward: -27.700, steps: 40
Episode 11: reward: -23.900, steps: 40
Episode 12: reward: -27.800, steps: 40
Episode 13: reward: -23.900, steps: 40
Episode 14: reward: -23.900, steps: 40
Episode 15: reward: -27.100, steps: 40
